In [1]:
from transformers import AutoTokenizer

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "EbanLee/kobart-summary-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
tokens = {i: tokenizer.decode([i]) for i in range(tokenizer.vocab_size)}

for token_index, token_str in tokens.items():
    print(f"{token_index}: '{token_str}'")

0: '<s>'
1: '</s>'
2: '<usr>'
3: '<pad>'
4: '<sys>'
5: '<unk>'
6: '<mask>'
7: '<unused0>'
8: '<unused1>'
9: '<unused2>'
10: '<unused3>'
11: '<unused4>'
12: '<unused5>'
13: '<unused6>'
14: '<unused7>'
15: '<unused8>'
16: '<unused9>'
17: '<unused10>'
18: '<unused11>'
19: '<unused12>'
20: '<unused13>'
21: '<unused14>'
22: '<unused15>'
23: '<unused16>'
24: '<unused17>'
25: '<unused18>'
26: '<unused19>'
27: '<unused20>'
28: '<unused21>'
29: '<unused22>'
30: '<unused23>'
31: '<unused24>'
32: '<unused25>'
33: '<unused26>'
34: '<unused27>'
35: '<unused28>'
36: '<unused29>'
37: '<unused30>'
38: '<unused31>'
39: '<unused32>'
40: '<unused33>'
41: '<unused34>'
42: '<unused35>'
43: '<unused36>'
44: '<unused37>'
45: '<unused38>'
46: '<unused39>'
47: '<unused40>'
48: '<unused41>'
49: '<unused42>'
50: '<unused43>'
51: '<unused44>'
52: '<unused45>'
53: '<unused46>'
54: '<unused47>'
55: '<unused48>'
56: '<unused49>'
57: '<unused50>'
58: '<unused51>'
59: '<unused52>'
60: '<unused53>'
61: '<unused54>'
62: